<a href="https://colab.research.google.com/github/MarcosVeniciu/AEDs/blob/master/Modelos_de_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

In [10]:
image_size = (180, 180)
input_shape = image_size + (3,)
num_classes = 2
dropout = 0.5

In [4]:
# Aplicando algumas transformações nas imagens, para aumentar o dataset

data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.12),
    ]
)

#Xception

In [23]:
def xception(input_shape, num_classes, dropout):
  base_model = keras.applications.Xception(
    weights=None,  # Os pesos serão inicializados aleatóriamente.
    input_shape=input_shape,
    include_top=False, # Não vai incluir a camada de classificação
  )  

  # freeze the base_model
  base_model.trainable = True # Sendo True vai permitir que os pesos sejam atualizados durante o treinamento

  # Create new model on top
  inputs = keras.Input(shape=input_shape)
  x = data_augmentation(inputs)  # Apply random data augmentation

  # Pre-trained Xception weights requires that input be scaled
  # from (0, 255) to a range of (-1., +1.), the rescaling layer
  # outputs: `(inputs * scale) + offset`
  scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
  x = scale_layer(x)
  
  # The base model contains batchnorm layers. We want to keep them in inference mode
  # when we unfreeze the base model for fine-tuning, so we make sure that the
  # base_model is running in inference mode here.
  x = base_model(x, training=True)
  x = keras.layers.GlobalAveragePooling2D()(x)

  if num_classes == 2:
    activation = "sigmoid"
    units = 1
  else:
    activation = "softmax"
    units = num_classes

  x = keras.layers.Dropout(dropout, name="dropout_2")(x) # Regularize with dropout
  x = keras.layers.Dense(1024, activation=activation)(x)
  x = keras.layers.Dense(512, activation=activation)(x)
  outputs = keras.layers.Dense(units, activation=activation)(x)
  return keras.Model(inputs, outputs)

In [24]:
modelo = xception(input_shape, num_classes, dropout)  
modelo.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 180, 180, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling_8 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 xception (Functional)       (None, 6, 6, 2048)        20861480  
                                                                 
 global_average_pooling2d_8   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                           

# MobileNetV2

In [25]:
def MobileNetV2(input_shape, num_classes, dropout):
  base_model = keras.applications.MobileNetV2(
    weights= None,  # Os pesos serão inicializados aleatóriamente.
    input_shape= input_shape,
    alpha=1.0,
    include_top=False, # Não vai incluir a camada de classificação.
  )

  # Freeze the base_model
  base_model.trainable = True # Sendo True vai permitir que os pesos sejam atualizados durante o treinamento

  inputs = keras.Input(shape = input_shape)
  x = data_augmentation(inputs)  # Apply random data augmentation

  scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
  x = scale_layer(x)

  x = base_model(x, training=True)
  x = keras.layers.GlobalAveragePooling2D()(x)

  if num_classes == 2:
    activation = "sigmoid"
    units = 1
  else:
    activation = "softmax"
    units = num_classes

  x = keras.layers.Dropout(dropout, name="dropout_2")(x) # Regularize with dropout
  x = keras.layers.Dense(640, activation=activation)(x)
  x = keras.layers.Dense(320, activation=activation)(x)
  outputs = keras.layers.Dense(units, activation=activation)(x)
  
  return keras.Model(inputs, outputs)  

In [26]:
modelo = MobileNetV2(input_shape, num_classes, dropout)  
modelo.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 180, 180, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling_9 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 mobilenetv2_1.00_180 (Funct  (None, 6, 6, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_9   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 1280)              0   

# Referências

[1] [Lista de modelos disponiveis no keras](https://keras.io/api/applications/#usage-examples-for-image-classification-models)